In [ ]:
import math
import os
import sys
import time

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random as rnd
import seaborn as sns
import scipy.stats as stats

from tqdm import tqdm
import statsmodels
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.power import TTestIndPower
from datetime import datetime, timedelta, date
from collections import namedtuple




import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import itertools
import warnings

from typing import Optional, List
from matplotlib.ticker import FuncFormatter
from scipy.stats import t, ttest_ind_from_stats
from scipy import stats
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.multitest import fdrcorrection



# Технические функции

In [ ]:
from libs.connections import sogu, anl # для подключения к продовому clickhouse и аналитическому postgresql

def binary_search(low, high, need_more_func):
    tolerance = 1e-4
    while high - low > tolerance:
        answer = (low + high) / 2
        if need_more_func(answer):
            low = answer
        else:
            high = answer
    return answer

from libs.sql import literal

def power_proportion(n0, k0, mde):
    p0 = k0 / n0
    p1 = p0 * (1 + mde/100)
    A = stats.binom(n0, p0)
    B = stats.binom(n0, p1)
    A_perc95 = A.ppf(0.95)
    return 1 - B.cdf(A_perc95)

def mde_proportion(n0, k0, alpha=0.05, beta=0.20):
    mde=50
    delta_mde = 25
    for _ in range(18): # 18 шагов достаточно
        pow = power_proportion(n0, k0, mde)
        dlt = pow - (1-beta)
        mde += delta_mde * (-1 if dlt > 0 else +1)
        delta_mde *= 2/3
    return mde

mde_proportion(146544, 16719)

1.8052720360899692

In [ ]:
class WHERE:
    """ 
        Cрез пользователей эксперимента
    """
    def __init__(self, where: str):
        self.where = where

    def __repr__(self):
        return self.where

class STAGE:
    """
        Числитель или знаменатель метрики 
    """
    def __init__(self, name:str, 
                       where:str, 
                       select:str, 
                randomization_unit_id:str='device_id', 
                database:str='clickhouse', 
                table:str='sogu.events', ):
        assert randomization_unit_id in ('device_id', 'order_id', 'email', )
        assert database in ('clickhouse', 'postgresql')
        assert table in ('sogu.events', 'analytics.mart_orders', )
        self.name = name
        self.where = where 
        self.select = select 
        self.randomization_unit_id = randomization_unit_id
        self.database = database
        self.table = table

    def __repr__(self):
        return self.name

stages_all = {
    #'device_id': STAGE("device_id", """ event_name = 'experiments.hit' and JSONExtract(params, 'experiment') = '{EXPERIMENT_NAME}' """, 'device_id'), 
    'device_id': STAGE("device_id", """ 1=1 """, 'device_id'),

    'main_pageview': STAGE("main_pageview", """ event_name = 'main.pageview' """, 'device_id'),
    'listing_pageview': STAGE("listing_pageview", """ event_name = 'listing.pageview' """, 'device_id'),
    'booking_pageview': STAGE("booking_pageview", """ event_name = 'booking.pageview' """, 'device_id'),

    #'header:top_header:app_download.shown': STAGE('header:top_header:app_download.shown', """ event_name = 'header:top_header:app_download.shown'""", 'device_id'),
    #'floating_bottom_button:app_download.shown': STAGE('floating_bottom_button:app_download.shown', """ event_name = 'floating_bottom_button:app_download.shown' """, 'device_id'),
    #'any_variant_app_download.shown': STAGE('any_variant_app_download.shown', """ event_name in ('header:top_header:app_download.shown', 'floating_bottom_button:app_download.shown') """, 'device_id'),
    #'any_variant_app_download.click': STAGE('any_variant_app_download.click', """ event_name in ('floating_bottom_button:app_download.click', 'header:top_header:app_download.click') """, 'device_id'),


    'sogu_order_created': STAGE('sogu_order_created', """ event_name = 'booking:order-create.success' """, 'device_id'),
    
    "anl_order_created": STAGE("anl_order_created", '1=1', '1', database="postgresql", table="analytics.mart_orders"),
    "order_w_paid": STAGE("order_w_paid", '1=1', "payment_date is not null", database="postgresql", table="analytics.mart_orders"),
    "order_w_refund": STAGE("order_w_refund", '1=1', "money_refund_date is not null", database="postgresql", table="analytics.mart_orders"),
    "delta_hours_order_to_date_exact": STAGE("delta_hours_order_to_date_exact", '1=1', "delta_hours_order_to_date_exact", database="postgresql", table="analytics.mart_orders"),
    "comission_tripster_final": STAGE("comission_tripster_final", "1=1", "comission_tripster_final", database="postgresql", table="analytics.mart_orders"),

}

class METRIC:
    def __init__(self, denominator, numerator, metric_type):
        assert denominator in stages_all, "Знаменателя метрики нет в стейджах"
        assert numerator in stages_all, "Числителя метрики нет в стейджах"
        assert metric_type in ('ratio', 'mean', 'user_based', 'binary'), "Укажите верный тип метрики"
        self.denominator = stages_all[denominator]
        self.numerator = stages_all[numerator]
        self.metric_type = metric_type

    def __repr__(self):
        return str((self.denominator.name, self.numerator.name, self.metric_type))
    
    def get_stages_set(self):
        return set([self.denominator, self.numerator])

metrics_all = {
    'Конверсия из девайса в просмотр главной': METRIC('device_id', 'main_pageview', 'ratio'),
    'Конверсия из девайса в просмотр листинга': METRIC('device_id', 'listing_pageview', 'ratio'),
    'Среднее количесто листингов на пользователя': METRIC('device_id', 'listing_pageview', 'mean'),

    'Конверсия из девайса в показ top_header:app_download': METRIC('device_id', 'header:top_header:app_download.shown', 'ratio'),
    'Конверсия из девайса в показ floating_bottom_button:app_download': METRIC('device_id', 'floating_bottom_button:app_download.shown', 'ratio'),

    'Конверсия из девайса в показ одного из двух app_download баннеров': METRIC('device_id', 'any_variant_app_download.shown', 'ratio'),
    'Конверсия из девайса в клик по одному из двух app_download баннеров': METRIC('device_id', 'any_variant_app_download.click', 'ratio'),

    'Конверсия из девайса в страницу букинга': METRIC('device_id', 'booking_pageview', 'ratio'),
    'Конверсия из девайса в создание заказа': METRIC('device_id', 'sogu_order_created', 'ratio'),
    'Конверсия из заказа в оплату': METRIC('anl_order_created', 'order_w_paid', 'ratio'), 
    'Конверсия из заказа в отмену оплаты': METRIC('anl_order_created', 'order_w_refund', 'ratio'),
    'Часов между заказом и исполнением': METRIC('anl_order_created', 'delta_hours_order_to_date_exact', 'mean'),
    'Конверсия в заказ с возвратом из девайса с заходом на главную': METRIC('main_pageview', 'order_w_refund', 'binary'),
    'Конверсия из листинга в оплаченный заказ': METRIC('listing_pageview', 'order_w_paid', 'binary'),
}

In [94]:
def get_all_stages_set(config):
    result_set = set()
    for metric_name in config['metrics']:
        result_set |= metrics_all[metric_name].get_stages_set()
    return result_set

def exp_days_from_config(config):
    start = datetime.fromisoformat(config['start_datetime'])
    finish = datetime.fromisoformat(config['finish_datetime'])
    return (finish - start).days + 1

exp_days_from_config(config)

2

In [17]:
from tqdm import tqdm

N = 100
real_data = pd.DataFrame({'order': np.random.randint(0, 2, size=N),
              ''
              'AB': np.random.choice(['A', 'B'], size=N),
              })
real_data['revenue'] = real_data['order'].apply(lambda x: 10 + (2.1 if x else 1.5) * np.random.normal())

display(real_data)

class MDE:
    def __init__(self, 
                 data: pd.DataFrame,
                 calculus_format: str = 'all_columns',
                 goal_metric: str = '',
                 metrics_columns: List[str] = [],
                 days_interval: int = np.nan,
                 alpha: float = 0.05,      
                 beta: float = 0.20,   
                 infinity: int = 1_000,
                 ):
        self.data = data 
        self.alpha = alpha
        self.beta = beta
        self.infinity = 1_000
        self.method_list = ('proportion_z_test', 'xi_2', '2 histograms', 'bootstrap')
    
        if days_interval is np.nan:
            print('Воспринимаем данные для расчёа mde как за 7 дней')
            days_interval = 7
        self.days_interval = days_interval

        if not metrics_columns:
            for column in data.columns:
                if real_data[column].dtype in (int, float):
                    metrics_columns.append(column)
                    continue
                print('Столбец', column, "не участвует в расчёте mde")
        print('Для расчёта MDE используем столбцы')
        print(metrics_columns)
        self.metrics_columns = metrics_columns

        if not goal_metric:
            goal_metric = metrics_columns[0]
        print('Целевая метрика:', goal_metric)
        self.goal_metric = goal_metric
    

    def calculate_one_mde_conversion(self, n_a, n_b, p_a, test_type='two-sided', method:str='proportion_z_test') -> float:
        """
        Выдаёт одно значние: относительный mde для конверсии
        При данных в группе А n_a - размер группы А, p_a - конверсия в группе А.
        n_b - размер группы B
        """
        if method == 'proportion_z_test':
            z_critical = stats.norm.ppf(1 - self.alpha/(2 if test_type == 'two-sided' else 1))
            alternative = 'smaller' if test_type == 'one-sided' else 'two-sided'

            p_b = binary_search(
                p_a, 1,
                lambda p_b: stats.proportions_ztest([n_a*p_a, n_b*p_b], [n_a, n_b], alternative=alternative)[0] > z_critical
            )
            return p_b / p_a - 1
        
        if method == 'histograms':
            assert 1==0, 'Метод histograms пока не реализован'

        if method == 'xi_2':
            assert 1==0, 'Метод xi_2 пока не реализован'

        if method == 'bootstrap':
            assert 1==0, 'Метод bootstrap пока не реализован'
        
        assert 1==0, f'Введён неподдерживаемй метод расчёта mde конверсии {method}. Выберите один из {self.method_list}'

    def calculate_list_mde_conversion(self, n_a_list:List[int], p_a, ratio:float=1.0, test_type='two-sided', method:str='proportion_z_test') -> List[float]:
        """
            ratio - во сколько раз тестовая группа больше контрольной
            n_a_list - массив количеств размеров выборок А
            p_a - конверсия в группе А
        """
        return [
            self.calculate_one_mde_conversion(n_a, int(n_a*ratio), p_a, test_type=test_type, method=method)
            for n_a in n_a_list]

    def calculate_one_mde_general(self, group_a:List[float], nobs1:int, ratio:float=1.0, test_type:str='two-sided', method:str='bootstrap' ) -> float: 
        """
            Выдаёт одно значние: относительный mde в общем случае любой метрики
            group_a - исторические данные группы А для определения mean и std
            nobs1 - объём реальной группы А. (объём исторических данных не совпадает объёмом настоящей выборки)
            ratio - во сколько раз тестовая группа больше контрольной

        """
        if method == 'TTestIndPower':
            mean_a = np.mean(group_a)
            std_a = np.std(group_a)
            mde_std = TTestIndPower().solve_power(nobs1=nobs1, alpha=self.alpha, power=1-self.beta, ratio=ratio, alternative=test_type)
            return (mde_std * std_a) / mean_a
        
        if method == 'bootstrap':
            def get_group_b_from_group_a(group_a:List[float], uplift:float) -> List[float]:
                # возможно, нужна более интеллектуальная трансформация
                group_b = np.array(group_a) * (1+uplift)
                return group_b
            
            def get_one_pvalue(group_a, group_b):
                """  
                    Расчитывает одно pvalue в симуляции
                """
                A = np.random.choice(group_a, size=nobs1)
                B = np.random.choice(group_b, size=int(nobs1*ratio))
                return stats.ttest_ind(A, B, equal_var=False, alternative=test_type)[1]

            def get_error_type2(uplift):
                """
                    Вероятность совершить ошибку II рода при данном uplift
                """
                group_b = get_group_b_from_group_a(group_a, uplift)
                pvalues = np.array([get_one_pvalue(group_a, group_b) for _ in range(self.infinity)])
                return (pvalues > self.alpha).mean()
            mde = binary_search(0, 0.8, need_more_func=lambda uplift: get_error_type2(uplift) > self.beta)

            def rnd_choice(M, size):
                return [np.mean(np.random.choice(M, size=size))
                        for _ in range(self.infinity)
                        ]

            plt.hist(rnd_choice(group_a, nobs1), color='green')
            plt.hist(rnd_choice(get_group_b_from_group_a(group_a, mde), int(nobs1*ratio)), color='red')
            return 


        assert 1==0, f'Введён неподдерживаемй метод расчёта mde конверсии {method}. Выберите один из {self.method_list}'

    def real_sampling(self):
        ''' 
            Симулирует бутстрапом истинное распределение pvalue всех метрик
        '''
        def one_real_sampling():
            sample1 = self.data[self.metrics_columns].sample(n=len(self.data), replace=True)
            sample2 = self.data[self.metrics_columns].sample(n=len(self.data), replace=True)
            #display(sample1.head(2))
            _, p_value_arr = stats.ttest_ind(sample1, sample2, equal_var=True, alternative='two-sided')
            return p_value_arr
        
        p_values = pd.DataFrame([one_real_sampling() for _ in range(self.infinity)], columns=self.metrics_columns)
        for column in self.metrics_columns:
            p_values[column+'_statistical_significance'] = p_values[column] < self.alpha

        goal_metric_not_statistical_significance = p_values[p_values[self.goal_metric+'_statistical_significance'] == False]
        for column in self.metrics_columns:
            if not (column == self.goal_metric):
                _ = str(round(100*goal_metric_not_statistical_significance[column+'_statistical_significance'].mean()))
                print(f'Вероятность случайного прокраса, когда не красится целевая метрика {_}%')

        return p_values
        

class AB_calculus:
    def __init__(self, config):
        self.config = config 

    def calculus_mde_from_data(self, calculus_method: str ='real_sampling'):
        pass
        

    def run(self):
        # загрузка данных за период
        self.data = pd.DataFrame()

        # вычисление mde происходит в любом случае
        self.calculus_mde_from_data()

        # если это АБ тестт расчитывается pvalue
        self.postanalysis()

mde_object = MDE(real_data.query('AB == "A"'))
res = mde_object.real_sampling()
print(res[mde_object.goal_metric+'_statistical_significance'].sum())
res

,order,AB,revenue
0,0,B,12.161913
1,1,B,9.143100
2,0,A,10.833253
3,1,A,9.534112
4,1,A,9.218656
...,...,...,...
95,1,A,9.125337
96,0,A,10.086966
97,0,A,11.159645
98,1,A,10.890728


Воспринимаем данные для расчёа mde как за 7 дней
Столбец AB не участвует в расчёте mde
Для расчёта MDE используем столбцы
['order', 'revenue']
Целевая метрика: order
Вероятность случайного прокраса, когда не красится целевая метрика 5%
63


,order,revenue,order_statistical_significance,revenue_statistical_significance
0,0.163603,0.159867,False,False
1,0.695053,0.938945,False,False
2,0.692791,0.120111,False,False
3,0.115307,0.517973,False,False
4,0.072359,0.869135,False,False
...,...,...,...,...
995,1.000000,0.302469,False,False
996,0.238854,0.353261,False,False
997,0.695053,0.298740,False,False
998,0.843846,0.770448,False,False


In [ ]:
config = {
    # если техническое название эксперимента есть, то experiment_name > ""
    # иначе смотрим н
    'experiment_name': 'download_app_motivators',
    'experiment_group_stages': [
        'ranking_rpv_new_1', 
        'stage_b_name',
    ],
    'variants': {'top_banner':'A', 'bottom_banner':'B'},
    'start_datetime': '2025-09-29',
    'finish_datetime': '2025-10-03',
    'n_groups': 2,
    'split_size': (0.5, 0.5),
    'alpha': 0.05,
    'beta': 0.20,
    
    'where': [],

    'goal_metric': 'Конверсия из листинга в оплаченный заказ',

    'metrics': [
        'Конверсия из девайса в просмотр главной',
        'Конверсия из девайса в просмотр листинга',

        'Конверсия из девайса в показ top_header:app_download',
        'Конверсия из девайса в показ floating_bottom_button:app_download',

        'Конверсия из девайса в показ одного из двух app_download баннеров',
        'Конверсия из девайса в клик по одному из двух app_download баннеров',

        'Среднее количесто листингов на пользователя',
        'Конверсия из девайса в создание заказа',
        'Конверсия из заказа в оплату',
        'Конверсия из заказа в отмену оплаты',
        'Часов между заказом и исполнением',
    ],
}

In [351]:
def craft_sogu_sql_from_many_metrics(config):
    '''
        Создаёт SQL для вычисления группы метрик со знаменателем device_id
    '''
    START_DATETIME = config['start_datetime']
    FINISH_DATETIME = config['finish_datetime']

    FROM_SOGU_WHERE_DT = f""" 
from sogu.events as se
where toDate(se.dt) between toDate('{START_DATETIME}') and toDate('{FINISH_DATETIME}')
    and se.dt between '{START_DATETIME}' and '{FINISH_DATETIME}'
    and se.device_id <> ''
    and ua_is_bot = 0 
"""
    for metric in config['metrics']:
        print(metrics_all[metric].denominator.name)
    metrics_list = [metrics_all[metric] for metric in config['metrics'] if metrics_all[metric].denominator.name == "device_id"]
    print(metrics_list)
    #for metric in config['metrics']:
    #    assert metrics_all[metric].denominator == metrics_all[config['metrics'][0]].denominator
        

    SELECT_LIST = []
    for metric in metrics_list:
        if metric.metric_type == 'ratio':
            st = f'anyIf(1, {metric.numerator.where}) as "has_{metric.numerator.name}"'
            #print(st)
            SELECT_LIST.append(st)
        if metric.metric_type == 'mean':
            st = f'countIf(1, {metric.numerator.where}) as "avg_{metric.numerator.name}"'
            #print(st)
            SELECT_LIST.append(st)
    #print(SELECT_LIST)

    if not config['experiment_name']:
        limitation_of_the_experiment_name = ''
    else:
        limitation_of_the_experiment_name = f''',
limitation_of_the_experiment_name as (
\tselect device_id 
\t{FROM_SOGU_WHERE_DT}
\t\tand event_name = 'experiments.hit' and JSONExtractString(params, 'experiment') = '{config['experiment_name']}'
),        
'''

    SELECT = '\n\t,'.join(SELECT_LIST)

    query = f'''
with orders as (
    select distinct 
        device_id
        ,JSONExtractInt(params, 'order_id') as order_id

\t\t{FROM_SOGU_WHERE_DT}
        and event_name = 'booking:order-create.success'
){limitation_of_the_experiment_name}

main as (
select 
\tdevice_id
\t,maxIf(JSONExtractString(params, 'variant'), event_name = 'experiments.hit' and JSONExtractString(params, 'experiment') = '{config['experiment_name']}') as ab_variant_min
\t,minIf(JSONExtractString(params, 'variant'), event_name = 'experiments.hit' and JSONExtractString(params, 'experiment') = '{config['experiment_name']}') as ab_variant_max
\t,ab_variant_min as ab_variant
\t,{SELECT}
{FROM_SOGU_WHERE_DT}
\tand {config['where'][0] if config['where'] else ' 1=1'}
\tand {'device_id in limitation_of_the_experiment_name' if limitation_of_the_experiment_name > '' else '1=1'}
group by device_id
having ab_variant_min = ab_variant_max
)
select 
    main.*
    ,orders.order_id
from main
left join orders
using device_id 


'''
    return query
st = craft_sogu_sql_from_many_metrics(config)
for e in st.split('\n'):
    print(e)
sogu_stages = sogu(st)

device_id
device_id
device_id
device_id
device_id
device_id
device_id
device_id
anl_order_created
anl_order_created
anl_order_created
[('device_id', 'main_pageview', 'ratio'), ('device_id', 'listing_pageview', 'ratio'), ('device_id', 'header:top_header:app_download.shown', 'ratio'), ('device_id', 'floating_bottom_button:app_download.shown', 'ratio'), ('device_id', 'any_variant_app_download.shown', 'ratio'), ('device_id', 'any_variant_app_download.click', 'ratio'), ('device_id', 'listing_pageview', 'mean'), ('device_id', 'sogu_order_created', 'ratio')]

with orders as (
    select distinct 
        device_id
        ,JSONExtractInt(params, 'order_id') as order_id

		 
from sogu.events as se
where toDate(se.dt) between toDate('2025-09-29') and toDate('2025-10-03')
    and se.dt between '2025-09-29' and '2025-10-03'
    and se.device_id <> ''
    and ua_is_bot = 0 

        and event_name = 'booking:order-create.success'
),
limitation_of_the_experiment_name as (
	select device_id 
	 
from

In [352]:
orders = list( set(sogu_stages.order_id.unique()) - set([0]) )
print(len(orders))

7371


In [353]:
for stage in get_all_stages_set(config):
    if stage.table == "analytics.mart_orders":
        print(stage.name, stage.table)

order_w_refund analytics.mart_orders
delta_hours_order_to_date_exact analytics.mart_orders
order_w_paid analytics.mart_orders
anl_order_created analytics.mart_orders


In [354]:
def craft_mart_orders_sql_from_many_metrics(config, orders):
    '''
        Создаёт SQL для вычисления группы метрик со знаменателем device_id
    '''
    # выбираем все стейджи таблицы analytics.mart_orders из всех метрик
    stage_list = [stage for stage in get_all_stages_set(config) if stage.table == "analytics.mart_orders"]
    print(stage_list)
        

    SELECT_LIST = [stage.select + ' as ' + stage.name for stage in stage_list]
    print(SELECT_LIST)

    SELECT = '\n\t,'.join(SELECT_LIST)

    #print(SELECT_LIST)
    query = f'''
select
    order_id
\t,{SELECT}
from analytics.mart_orders
where order_id in ({literal(orders)})
    '''
    return query

st = craft_mart_orders_sql_from_many_metrics(config, orders)
for e in st.split('\n'):
    print(e)
    
mart_orders_stages = anl(st)

[order_w_refund, delta_hours_order_to_date_exact, order_w_paid, anl_order_created]
['money_refund_date is not null as order_w_refund', 'delta_hours_order_to_date_exact as delta_hours_order_to_date_exact', 'payment_date is not null as order_w_paid', '1 as anl_order_created']

select
    order_id
	,money_refund_date is not null as order_w_refund
	,delta_hours_order_to_date_exact as delta_hours_order_to_date_exact
	,payment_date is not null as order_w_paid
	,1 as anl_order_created
from analytics.mart_orders
where order_id in (5701634,5701635,5701636,5701638,5701639,5701640,5701641,5701642,5701646,5701648,5701649,5701650,5701651,5701653,5701656,5701657,5701658,5701659,5701660,5701661,5701662,5701664,5701666,5701667,5701669,5701670,5701671,5701675,5701676,5701678,5701680,5701682,5701683,5701685,5701689,5701690,5701691,5701695,5701696,5701702,5701704,5701706,5701707,5701709,5701710,5701714,5701716,5701717,5701719,5701720,5701721,5701722,5701723,5701724,5701726,5701727,5701728,5701729,5701731

In [319]:
main = sogu_stages.merge(mart_orders_stages, how='left', on='order_id')
display(main.head(2))


stage_columns = [col for col in set(main.columns) if col not in ('device_id', 'order_id')]
main_gr = main.groupby('device_id').agg({col: 'max' for col in stage_columns}).reset_index() \
        .query('has_listing_pageview == 1') \
        .fillna(0)
#main_gr['AB'] = np.random.choice(['A', 'B'], size=len(main_gr))
main_gr['AB'] = main_gr.ab_variant.apply(lambda st: config['variants'][st])
main_gr.head(2)

,device_id,ab_variant_min,ab_variant_max,ab_variant,has_main_pageview,has_listing_pageview,has_header:top_header:app_download.shown,has_floating_bottom_button:app_download.shown,has_any_variant_app_download.shown,has_any_variant_app_download.click,avg_listing_pageview,has_sogu_order_created,order_id,order_w_refund,delta_hours_order_to_date_exact,order_w_paid,anl_order_created
0,122f5064-381b-4a17-9016-8a2253028f0a,bottom_banner,bottom_banner,bottom_banner,0,1,0,1,1,1,8,0,0,NaN,NaN,NaN,NaN
1,ae9c8768-04e6-4be4-98c9-453fbd8de6fe,bottom_banner,bottom_banner,bottom_banner,0,1,0,0,0,0,1,0,0,NaN,NaN,NaN,NaN


,device_id,has_sogu_order_created,avg_listing_pageview,ab_variant_min,has_any_variant_app_download.shown,order_w_paid,delta_hours_order_to_date_exact,anl_order_created,has_floating_bottom_button:app_download.shown,has_main_pageview,has_listing_pageview,has_header:top_header:app_download.shown,ab_variant_max,order_w_refund,ab_variant,has_any_variant_app_download.click,AB
0,00008627-1bcd-4978-a2da-293107c093f1,0,1,top_banner,1,0,0.0,0.0,0,0,1,1,top_banner,0,top_banner,0,A
1,00011e3f-f51f-49d2-a1ff-9377a5117c1a,0,1,bottom_banner,1,0,0.0,0.0,1,0,1,0,bottom_banner,0,bottom_banner,0,B


In [320]:
stage_columns

['has_sogu_order_created',
 'avg_listing_pageview',
 'ab_variant_min',
 'has_any_variant_app_download.shown',
 'order_w_paid',
 'delta_hours_order_to_date_exact',
 'anl_order_created',
 'has_floating_bottom_button:app_download.shown',
 'has_main_pageview',
 'has_listing_pageview',
 'has_header:top_header:app_download.shown',
 'ab_variant_max',
 'order_w_refund',
 'ab_variant',
 'has_any_variant_app_download.click']

In [355]:
def calculus(df):
    """
        Расчёт всех метрик в датафрейме с pvalue
    """
    print("Соотношение uid A на B", len(df.query('AB == "A"')), len(df.query('AB == "B"')))

    metrics = list(set(stage_columns) - set(['ab_variant_min', 'ab_variant_max', 'ab_variant']))
    gr = df.groupby('AB').agg({m: list for m in metrics})

    gr.loc['pv'] = [round(stats.ttest_ind(gr[m]['A'], gr[m]['B']).pvalue, 3) for m in metrics]
    gr.loc['delta'] = round((gr.iloc[1].apply(lambda x: np.mean(x)) / gr.iloc[0].apply(lambda x: np.mean(x)) - 1)*100, 1)
    #gr.rename(columns={m: 'uid2'+m for m in metrics_uid2}
    #                 | {m: m+' per_uid' for m in metrics_per_uid}, inplace=True)

    T = gr.T
    T['delta'] = T['delta'].apply(lambda x: str(x)+'%' if x<0 else f'+{str(x)}%')
    T['A'] = T['A'].apply(lambda x: round(np.mean(x), 4))
    T['B'] = T['B'].apply(lambda x: round(np.mean(x), 4))
    return T[['A', 'B', 'delta', 'pv']]


res = calculus(main_gr)
res.index = [(reverse_dict[e.replace('has_', "")] if e.replace('has_', "") in reverse_dict else e) for e in res.index]
res.index = [(reverse_dict[e.replace('avg_', "")] if e.replace('avg_', "") in reverse_dict else e) for e in res.index]
res
res

Соотношение uid A на B 58030 57353


AB,A,B,delta,pv
Конверсия из девайса в создание заказа,0.0412,0.0418,+1.3%,0.636
Среднее количесто листингов на пользователя,3.4107,3.3427,-2.0%,0.836
Конверсия из листинга в оплаченный заказ,0.0235,0.0241,+2.6%,0.498
Конверсия из девайса в показ одного из двух app_download баннеров,0.9287,0.3497,-62.3%,0.0
Часов между заказом и исполнением,11.1706,10.9626,-1.9%,0.764
anl_order_created,0.0359,0.0360,+0.5%,0.883
Конверсия из девайса в показ floating_bottom_button:app_download,0.0000,0.3497,+1014587.7%,0.0
Конверсия из девайса в просмотр главной,0.1549,0.1549,+0.0%,0.975
Среднее количесто листингов на пользователя,1.0000,1.0000,+0.0%,NaN
Конверсия из девайса в показ top_header:app_download,0.9287,0.0000,-100.0%,0.0


In [341]:
st = 'has_any_variant_app_download.click'.replace('has_', "")
st

'any_variant_app_download.click'

In [ ]:
reverse_dict

In [345]:
str(metrics_all[e].numerator)

'order_w_paid'

In [346]:
reverse_dict = {}
for e in metrics_all:
    reverse_dict[str(metrics_all[e].numerator)] = e
reverse_dict

{'main_pageview': 'Конверсия из девайса в просмотр главной',
 'listing_pageview': 'Среднее количесто листингов на пользователя',
 'header:top_header:app_download.shown': 'Конверсия из девайса в показ top_header:app_download',
 'floating_bottom_button:app_download.shown': 'Конверсия из девайса в показ floating_bottom_button:app_download',
 'any_variant_app_download.shown': 'Конверсия из девайса в показ одного из двух app_download баннеров',
 'any_variant_app_download.click': 'Конверсия из девайса в клик по одному из двух app_download баннеров',
 'booking_pageview': 'Конверсия из девайса в страницу букинга',
 'sogu_order_created': 'Конверсия из девайса в создание заказа',
 'order_w_paid': 'Конверсия из листинга в оплаченный заказ',
 'order_w_refund': 'Конверсия в заказ с возвратом из девайса с заходом на главную',
 'delta_hours_order_to_date_exact': 'Часов между заказом и исполнением'}

In [139]:
def calculus_mde_for_goal_metric(main_gr, config):
    """
        По датафрейму main_gr вычисляет mde для целевой метрики
    """
    days_exp = exp_days_from_config(config)
    goal_metric = metrics_all[config['goal_metric']]
    print(goal_metric)
    print(goal_metric.denominator)
    print(goal_metric.numerator)
    result = pd.DataFrame()
    df = main_gr[main_gr['has_'+str(goal_metric.denominator)].astype(bool)]
    n0 = len(df) // 2
    p0 = df[str(goal_metric.numerator)].mean()
    print(n0, p0)
    for days_mde in [7, 14, 21]:
        n_mde = int(days_mde * (n0//days_exp))
        mde = round(mde_proportion(n_mde, int(n_mde*p0)), 2)
        new_row = pd.DataFrame({'n_one_group': [n_mde, ], 'days': [days_mde, ], 'mde': [mde, ], 'abs_mde': [str(round(p0*mde, 3))+'%'], })
        result = pd.concat([result, new_row])
    result['n_one_group'] = result['n_one_group'].astype(int)
    result['days'] = result['days'].astype(int)
    print(f'{round(p0*100, 2)}% =', config['goal_metric'],)
    return result

calculus_mde_for_goal_metric(main_gr, config)

('listing_pageview', 'order_w_paid', 'binary')
listing_pageview
order_w_paid
26351 0.03096597916627137
3.1% = Конверсия из листинга в оплаченный заказ


,n_one_group,days,mde,abs_mde
0,92225,7,4.64,0.144%
0,184450,14,3.27,0.101%
0,276675,21,2.67,0.083%


In [111]:
mde_proportion(52703, 5712)

go mde_proportion 52703 5712


3.14000784558403

In [112]:
mde_proportion(553381, 17136)

go mde_proportion 553381 17136


1.8877374732754557

In [91]:
config

{'start_datetime': '2025-08-01',
 'finish_datetime': '2025-08-02',
 'n_groups': 2,
 'split_size': (0.5, 0.5),
 'alpha': 0.05,
 'beta': 0.2,
 'where': [ua_is_mobile = 1],
 'goal_metric': 'Конверсия из листинга в оплаченный заказ',
 'metrics': ['Конверсия из девайса в просмотр главной',
  'Конверсия из девайса в просмотр листинга',
  'Среднее количесто листингов на пользователя',
  'Конверсия из девайса в создание заказа',
  'Конверсия из заказа в оплату',
  'Конверсия из заказа в отмену оплаты',
  'Часов между заказом и исполнением']}

array([1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1])

In [50]:
main_gr.agg({col: 'mean' for col in stage_columns})

has_sogu_order_created             0.040908
avg_listing_pageview               5.008368
order_w_paid                       0.030966
delta_hours_order_to_date_exact    6.945601
anl_order_created                  0.040908
has_main_pageview                  0.121132
has_listing_pageview               1.000000
order_w_refund                     0.001727
dtype: float64

In [ ]:




class ExperimentReport:
    def __init__(
        self, 
        df: pd.DataFrame, 
        m_params: dict,
        exp_params: dict
    ):
        """
        Инициализирует объект класса FixedHorizon.

        Параметры:
            df (pd.DataFrame): DataFrame с данными.
            m_params (dict): Параметры метрик.
            exp_params (dict): Параметры дизайна эксперимента и расчетов
        """
        assert isinstance(df, pd.DataFrame), "df должен быть DataFrame"
        assert isinstance(m_params, dict), "m_params должен быть словарем"
        assert isinstance(exp_params, dict), "exp_params должен быть словарем"

        self.df = df
        self.m_params = m_params
        self.alpha = exp_params['alpha']
        self.beta = exp_params['beta']
        self.date_col = exp_params['date_col']
        self.split_col = exp_params['split_col']
        self.rolling = exp_params['rolling']

    @staticmethod
    def calculate_ratio_variance(numerator: float, denominator: float, is_same_den_n_col: bool) -> float:
        """
        Получение дисперсии для метрики Delta-method'ом.

        Параметры:
            numerator (float): Числитель.
            denominator (float): Знаменатель.
            is_same_den_n_col (bool): Флаг, указывающий, одинаковы ли знаменатель и n_col.

        Возвращает:
            float: Вычисленное значение дисперсии.
        """
        try:
            numerator_mean = np.mean(numerator)
            numerator_var = np.var(numerator)
            if is_same_den_n_col:
                denominator_mean = 1.0
                denominator_var = 0.0
            else:
                denominator_mean = np.mean(denominator)
                denominator_var = np.var(denominator)

            cov = np.mean((numerator - numerator_mean) * (denominator - denominator_mean))

            var = (numerator_var / denominator_mean ** 2
                + denominator_var * numerator_mean ** 2 / denominator_mean ** 4
                - 2 * numerator_mean / denominator_mean ** 3 * cov)
            return var
        except Exception as e:
            print(f"Произошла ошибка при расчете дисперсии: {e}")
            return float('nan')
    
    @staticmethod
    def convert_notna_as_int(func, value):
        if func == pd.Series.nunique:
            value = value.notna().astype(int)
        return value
    
    def calculate_metric_data(self, split_data, params):
        """
        Калькуляция статистик по метрике

        Параметры:
            split_data (pd.DataFrame): Датафрейм по 1 сплиту.
            params (dict): Словарь с параметрами метрики.

        Возвращает:
            - rto – среднее, отношение числителя к знаменателю
            - numerator_total – сумма по числителю
            - denominator_total – сумма по знаменателю
            - metric_var – дисперсия отношения случайных величин
            - metric_n – количество наблюдений в пользователях по метрике отношения
        """
        num_col, num_func = params["num"]
        den_col, den_func = params["den"]
        same_den_n_col_flg = True if den_col == params['n_col'] else False
        n_col = params["n_col"]

        numerator, denominator = split_data[num_col], split_data[den_col]

        numerator_total, denominator_total = num_func(numerator), den_func(denominator)

        rto = numerator_total / denominator_total

        metric_var = self.calculate_ratio_variance(
            self.convert_notna_as_int(num_func, numerator), 
            self.convert_notna_as_int(den_func, denominator), 
            same_den_n_col_flg
        )

        metric_n = split_data[n_col].nunique()

        return rto, numerator_total, denominator_total, metric_var, metric_n
    
    def calculate_stats(self, df: pd.DataFrame, max_date: Optional[str] = None) -> pd.DataFrame:
        """
        Калькуляция стат. значимости, FH по всем метрикам эксперимента

        Параметры:
            df (pd.DataFrame): Датафрейм по эксперименту.
            max_date (str): последняя дата на которую актуализируется отчет.

        Возвращает:
            pd.DataFrame – таблица с результатами по всем метрикам
        """
        df_ = df[(df[self.date_col] <= max_date)] if max_date is not None else df
        split_0, split_1 = [x for _, x in df_.groupby(self.split_col)]
        result = []

        for metric_name, params in self.m_params.items():
            rto_0, numerator_total_0, denominator_total_0, var_0, n_0 = self.calculate_metric_data(split_0, params)
            rto_1, numerator_total_1, denominator_total_1, var_1, n_1 = self.calculate_metric_data(split_1, params)
            std_0 = var_0**0.5
            std_1 = var_1**0.5

            # Вычисляем p-value с помощью ttest_ind_from_stats
            _, pvalue = ttest_ind_from_stats(mean1=rto_0, std1=std_0, nobs1=n_0, mean2=rto_1, std2=std_1, nobs2=n_1)
            
            # Fixed Horizon
            effect_size = abs(rto_0 - rto_1) / ((std_0**2 + std_1**2) / 2)**0.5
            power = tt_ind_solve_power(effect_size=effect_size, nobs1=n_0, alpha=0.05, ratio=n_1/n_0)
            n_need = int(tt_ind_solve_power(effect_size=effect_size, power=self.beta, alpha=self.alpha, ratio=n_1/n_0))
            mde = tt_ind_solve_power(nobs1=n_0, power=self.beta, alpha=self.alpha, ratio=n_1/n_0)

            result.append(pd.Series({
                'metric_name': metric_name,
                'value_0': rto_0,
                'value_1': rto_1,
                'numerator_name': params["num"][0],
                'denominator_name': params["den"][0],
                'numerator_value_0': numerator_total_0,
                'denominator_value_0': denominator_total_0,
                'numerator_value_1': numerator_total_1,
                'denominator_value_1': denominator_total_1,
                'lift': (rto_1 - rto_0) / rto_0,
                'delta': rto_1 - rto_0,
                'var_0': var_0,
                'n_0': n_0,
                'var_1': var_1,
                'n_1': n_1,
                'pvalue': pvalue,
                'power': power,
                'mde_fact': effect_size,
                'mde_estimated': mde,
                'n_estimated': n_need,
                'is_significant': pvalue < self.alpha,
                'is_fh': power > self.beta,
            }))
                    
        return pd.DataFrame(result)
    
    def calculate_split_pairs(self):
        combs = self.df.sort_values(by=self.split_col)[self.split_col].dropna().unique()
        return itertools.combinations(list(combs), 2)
    
    def calculate_report(self):
        """
        Получение калькуляций по эксперименту.

        Результат:
            Записывает DataFrame в атрибут self.report. 
            Столбцы: 
                date, split_pair, metric_name, value_0, value_1, numerator_name, numerator_value_*, denominator_name, denominator_value_*,
                lift, var_*, n_*, pvalue, pvalue_adj, power, mde_fact, mde_estimated, is_significant, is_significant_adj, is_fh
        """
        try:
            self.df[self.date_col] = pd.to_datetime(self.df[self.date_col])
            dt_min = self.df[self.date_col].min() if self.rolling else self.df[self.date_col].max()
            dt_max = self.df[self.date_col].max()
            date_range = pd.Series(pd.date_range(start=dt_min, end=dt_max), dtype=str).tolist() 

            result = []

            for pair in self.calculate_split_pairs():
                df_filtered = self.df[self.df[self.split_col].isin(pair)]
                for dt in date_range:
                    day_of_experiment = (pd.to_datetime(dt) - self.df[self.date_col].min()).days + 1
                    day_report = self.calculate_stats(df=df_filtered, max_date=dt)
                    day_report['date'] = dt
                    day_report['day_of_experiment'] = day_of_experiment
                    day_report['split_pair'] = '_'.join(str(i) for i in pair)
                    result.append(day_report)

            self.report = pd.concat(result)
            _, self.report['pvalue_adj'] = fdrcorrection(self.report['pvalue'])
            self.report.loc[:, 'is_significant_adj'] = self.report['pvalue_adj'] < self.alpha

            cols_to_move = ['date', 'day_of_experiment', 'split_pair']
            cols = self.report.columns.tolist()
            cols = cols_to_move + [col for col in cols if col not in cols_to_move]
            self.report = self.report[cols]
        except Exception as e:
            print(f"Произошла ошибка при calculate_report: {e}")

    def visualize_report(self, metric_names, split_pair):
        for metric_name in metric_names:
            filtered_data = self.report[(self.report['metric_name'] == metric_name) & 
                                        (self.report['split_pair'] == split_pair)]
            
            sns.set(style="whitegrid")
            fig, axes = plt.subplots(1, 5, figsize=(20, 4))

            axes[0].axis('off')
            summary = filtered_data.iloc[-1]

            text_content = (
                f"Название метрики: {summary['metric_name']}\n"
                f"Значения средних: {summary['value_0']:.2f}, {summary['value_1']:.2f}\n"
                f"Lift: {summary['lift']:.2f}\n"
                f"P-value (скорректированное): {summary['pvalue_adj']:.2f}\n"
                f"Мощность: {summary['power']:.2f}\n"
                f"Значимо: {'Да' if summary['is_significant_adj'] else 'Нет'}\n"
                f"Необходимо юзеров на 1 группу: {summary['n_estimated']:.2f}\n"
                f"Всего юзеров: {summary['n_0']:.2f}, {summary['n_1']:.2f}\n"
                f"Fixed Horizon: {'Да' if summary['is_fh'] else 'Нет'}"
            )

            axes[0].text(0, 1, text_content, verticalalignment='top', fontsize=10)

            # lift plot
            colors = ['red' if (lift < 0 and is_sig) else 'green' if (lift > 0 and is_sig) else 'black' 
                    for lift, is_sig in zip(filtered_data['lift'], filtered_data['is_significant_adj'])]
            sns.lineplot(x="day_of_experiment", y="lift", data=filtered_data, ax=axes[2], palette=colors, marker="o")
            axes[2].axhline(0, color='black', linestyle='--')
            axes[2].set_title(f'{metric_name}: lift over time')
            
            # means plot
            sns.lineplot(x="day_of_experiment", y="value_0", data=filtered_data, ax=axes[1], label='Value 0', color='blue')
            sns.lineplot(x="day_of_experiment", y="value_1", data=filtered_data, ax=axes[1], label='Value 1', color='orange')
            axes[1].set_title(f'{metric_name}: mean over time')
            
            # pvalue_adj plot
            colors_pvalue = ['red' if (pval < self.alpha and lift < 0) or (pval < self.alpha and lift > 0) else 'black' 
                            for pval, lift in zip(filtered_data['pvalue_adj'], filtered_data['lift'])]
            sns.lineplot(x="day_of_experiment", y="pvalue_adj", data=filtered_data, ax=axes[4], palette=colors_pvalue, marker="o")
            axes[4].axhline(self.alpha, color='black', linestyle='--')
            axes[4].set_title(f'{metric_name}: Adjusted p-value over time')
            
            # Power plot
            colors_power = ['red' if (power > self.beta and lift < 0) else 'green' if (power > self.beta and lift > 0) else 'black' 
                            for power, lift in zip(filtered_data['power'], filtered_data['lift'])]
            sns.lineplot(x="day_of_experiment", y="power", data=filtered_data, ax=axes[3], palette=colors_power, marker="o")
            axes[3].axhline(self.beta, color='black', linestyle='--')
            axes[3].set_title(f'{metric_name}: Power over time')
            

            plt.tight_layout()
            plt.show()

N = 100
df = pd.DataFrame({'order': np.random.randint(0, 2, size=N),
              'AB': np.random.choice(['A', 'B'], size=N),
              'revenue': np.random.normal(size=N),
              })

'''
        self.df = df
        self.m_params = m_params
        self.alpha = exp_params['alpha']
        self.beta = exp_params['beta']
        self.date_col = exp_params['date_col']
        self.split_col = exp_params['split_col']
        self.rolling = exp_params['rolling']
'''

m_params = {}
exp_params = {'alpha': 0.05,
              'beta': 0.20,
              'date_col': ['2025-03-01', '2025-03-10'],
              'split_col': 'AB',
              'rolling': 0.5,
              }
x = ExperimentReport(df, m_params, exp_params)
x

array([ 0.95799356,  0.0222412 ,  0.93487425,  0.39367996, -0.21258755,
        0.22915357, -0.82241128,  1.37313675, -0.03464981, -0.50074178,
       -0.8436727 ,  0.68398035, -2.04998491,  0.19584212, -0.47321695,
        1.15326596, -3.14287105,  0.81968176, -0.79725344, -0.29393408,
       -0.33463401, -0.82960345,  0.57910153,  0.59401951,  0.90170594,
       -0.24013569, -0.21825521, -0.04298059, -0.35100256,  0.79816606,
        0.07081019,  0.73750915, -0.63406898, -1.08946941,  0.28300168,
       -0.15992228, -0.84938514, -1.27369781, -1.28073783, -0.74578929,
       -0.07660019,  1.75807588, -1.20752286, -0.88386138, -0.61852534,
        1.29458864, -0.60775045,  1.5012997 , -0.39419634,  0.10157417,
       -0.9740476 ,  1.38427045,  1.09011003,  1.01173315,  0.9598419 ,
       -1.14624034, -0.31771389, -1.2681216 ,  1.87268345,  1.72491882,
        0.08804779, -1.08475819,  1.66066556, -1.35955537, -0.80305301,
        0.36392412, -0.14952173,  1.01099896,  0.04602495, -1.06

In [39]:
def sogu_filter_has_events(filt: str, events: list, params, query_show: bool = False):
    """ 
        Для device_id у которых выполняется условие filt выполняется 2 подсчёта:
        Находятся все созданные заказы 'booking:order-create.success'
        Подсчитывается факт наличия событий из списка events

        Итоговое количество строк в таблице = количество уникальных (device_id, order_id) если заказы были
        и (device_id, "" as order_id) если заказов не было

        Пример входных данных:
        filt = " event_name = 'main.pageview' "
        events = ['main.pageview', 'listing.pageview', 'booking:order-create.success']
    """

    FROM_SOGU_WHERE_DT = f'''from sogu.events
            where toDate(dt) between '{params['start_dt']}' and '{params['finish_dt']}'
    '''
    
    query = f''' 
            with devices as (
                select distinct 
                    device_id
                {FROM_SOGU_WHERE_DT}
                    and device_id <> '' 
                    and {filt}
            ), 
            ords as (
                select 
                    device_id
                    ,JSONExtractString(params, 'order_id') as order_id
                {FROM_SOGU_WHERE_DT}
                    and device_id in devices
                    and event_name = 'booking:order-create.success'
            )
            select
                se.device_id 
                ,ords.order_id
                ,{"""
                ,""".join([f"""if(countIf(event_name = '{en}') > 0, se.device_id, Null) as "has_{en}" """ 
                    for en in events
                ])}
            from sogu.events se

            left join ords
            on se.device_id = ords.device_id

            where toDate(se.dt) between '{params['start_dt']}' and '{params['finish_dt']}'
                and se.device_id <> ''  
                and event_name in ({literal(events)})
                and se.device_id in devices
            group by se.device_id, ords.order_id
        '''
    if query_show:
        print(query)
    '''
    for ev in df.event_name.unique():
        #print(ev)
        devises = set(df[df.event_name == ev].device_id)
        df['has_'+ev] = df['device_id'].isin(devises)
    '''
    return sogu(query)

In [ ]:
metrics_all = {
    "DEVICE_ID2MAIN_PAGEVIEW__metric": ("DEVICE_ID", 'conversion', "MAIN_PAGEVIEW",
    DEVICE_ID2SOGU_ORDER_CREATED__metric = "DEVICE_ID", 'conversion', "SOGU_ORDER_CREATED",
    DEVICE_ID2LISTING_PAGEVIEW__metric = "DEVICE_ID", 'conversion', "LISTING_PAGEVIEW",
}

In [ ]:
DEVICE_ID2MAIN_PAGEVIEW__metric = "DEVICE_ID", 'conversion', "MAIN_PAGEVIEW",
DEVICE_ID2SOGU_ORDER_CREATED__metric = "DEVICE_ID", 'conversion', "SOGU_ORDER_CREATED",
DEVICE_ID2LISTING_PAGEVIEW__metric = "DEVICE_ID", 'conversion', "LISTING_PAGEVIEW",
ORDER2REFUND = "DEVICE_ID", 'conversion', ""

In [ ]:
params = {
    'start_dt': '2025-08-01',
    'finish_dt': '2025-08-21',
}

In [ ]:
def sogu_filter_has_events(filt: str, events: list, params, query_show: bool = False):
    """ 
        Для device_id у которых выполняется условие filt выполняется 2 подсчёта:
        Находятся все созданные заказы 'booking:order-create.success'
        Подсчитывается факт наличия событий из списка events

        Итоговое количество строк в таблице = количество уникальных (device_id, order_id) если заказы были
        и (device_id, "" as order_id) если заказов не было

        Пример входных данных:
        filt = " event_name = 'main.pageview' "
        events = ['main.pageview', 'listing.pageview', 'booking:order-create.success']
    """

    FROM_SOGU_WHERE_DT = f'''from sogu.events
            where toDate(dt) between '{params['start_dt']}' and '{params['finish_dt']}'
    '''
    
    query = f''' 
            with devices as (
                select distinct 
                    device_id
                {FROM_SOGU_WHERE_DT}
                    and device_id <> '' 
                    and {filt}
            ), 
            ords as (
                select 
                    device_id
                    ,JSONExtractString(params, 'order_id') as order_id
                {FROM_SOGU_WHERE_DT}
                    and device_id in devices
                    and event_name = 'booking:order-create.success'
            )
            select
                se.device_id 
                ,ords.order_id
                ,{"""
                ,""".join([f"""if(countIf(event_name = '{en}') > 0, se.device_id, Null) as "has_{en}" """ 
                    for en in events
                ])}
            from sogu.events se

            left join ords
            on se.device_id = ords.device_id

            where toDate(se.dt) between '{params['start_dt']}' and '{params['finish_dt']}'
                and se.device_id <> ''  
                and event_name in ({literal(events)})
                and se.device_id in devices
            group by se.device_id, ords.order_id
        '''
    if query_show:
        print(query)
    '''
    for ev in df.event_name.unique():
        #print(ev)
        devises = set(df[df.event_name == ev].device_id)
        df['has_'+ev] = df['device_id'].isin(devises)
    '''
    return sogu(query)

In [104]:
def calculus(df, metrics = METRICS):
    print('devcieA', len(df.query('AB == "A"')), 'deviceB',  len(df.query('AB == "B"')))
    #print('orderA', len(df.query('AB == "A"').query('order == 1')), 'orderB',  len(df.query('AB == "B"').query('order == 1')))
    gr = df.groupby('AB').agg({m: list for m in metrics})
    gr.loc['pv'] = [round(stats.ttest_ind(gr[m]['A'], gr[m]['B']).pvalue, 3) for m in metrics]
    gr.loc['delta'] = round((gr.iloc[1].apply(lambda x: np.mean(x)) / gr.iloc[0].apply(lambda x: np.mean(x)) - 1)*100, 1)

    T = gr.T
    T['delta'] = T['delta'].apply(lambda x: '+inf%' if x > 500 else str(x)+'%' if x<0 else f'+{str(x)}%')
    T['A'] = T['A'].apply(lambda x: round(np.mean(x), 5))
    T['B'] = T['B'].apply(lambda x: round(np.mean(x), 5))
    #print(T['A']['uid2ok'], T['B']['uid2ok'])
    print(T.columns)
    #return T[T.AB != 'has_main.pageview'][['A', 'B', 'delta', 'pv']]
    return T[T.index != 'has_main.pageview'][['A', 'B', 'delta', 'pv']]


print('Весь тест')
x = calculus(df)
x

Весь тест
devcieA 146261 deviceB 146843
Index(['A', 'B', 'pv', 'delta'], dtype='object', name='AB')


AB,A,B,delta,pv
has_listing.pageview,0.53608,0.53699,+0.2%,0.62
has_registration.pageview,0.01395,0.01380,-1.0%,0.739
has_experience.pageview,0.49352,0.49420,+0.1%,0.71
has_booking-calendar.pageview,0.24472,0.24432,-0.2%,0.799
has_booking.pageview,0.24472,0.24432,-0.2%,0.799
has_booking:order-create.success,0.11503,0.11471,-0.3%,0.791
is_confirmed,0.04138,0.04074,-1.6%,0.377
full_price,743.70886,722.36440,-2.9%,0.325
delta_hours_order_to_reject,7.03391,7.14234,+1.5%,0.598
money_refund_amount_rub,14.22711,13.71977,-3.6%,0.604


In [96]:
events = ['main.pageview', 'listing.pageview', 'registration.pageview', 
                            'experience.pageview', 'booking-calendar.pageview', 'booking.pageview', 
                            'booking:order-create.success']
main = sogu_filter_has_events(" event_name = 'main.pageview' ", events, params)
print(len(main))
display(main.head(2))



310615


,device_id,order_id,has_main.pageview,has_listing.pageview,has_registration.pageview,has_experience.pageview,has_booking-calendar.pageview,has_booking.pageview,has_booking:order-create.success
0,87d5c4aa-b062-40f1-83b8-ceab49b30ee5,,87d5c4aa-b062-40f1-83b8-ceab49b30ee5,87d5c4aa-b062-40f1-83b8-ceab49b30ee5,None,None,None,None,None
1,b1de86be-b1d5-4df9-ad24-b2cfd6564c26,,b1de86be-b1d5-4df9-ad24-b2cfd6564c26,None,None,None,None,None,None


In [ ]:
metrincs_order = ['is_confirmed', 'full_price', 'delta_hours_order_to_reject', 'money_refund_amount_rub', 'comission_tripster_final']

order_ids = set(main.order_id) - set([''])

mart_orders = anl(f''' 
    select order_id, {' ,'.join(metrincs_order)}
    from analytics.mart_orders
    where order_id in ({literal(order_ids)})
        and guide_id <> 20329
    ''')

mart_orders['order_id'] = mart_orders['order_id'].astype(str)
print(len(mart_orders))
mart_orders.head(2)

51173


,order_id,is_confirmed,full_price,delta_hours_order_to_reject,money_refund_amount_rub,comission_tripster_final
0,5404912,True,150.0,NaN,NaN,3201.0
1,5499076,False,9000.0,NaN,NaN,2250.0


In [ ]:
experience_ids = set(main.order_id) - set([''])

mart_experience = anl(''' 
    select 
        experience_id
        ,city_name, country_name, region_name
        ,exp_type, exp_type_detailed, exp_formatl, movement_type, status
        ,price_category, pay_moment
        ,duration_class
    from analytics.mart_experience
    where experience_id in '{literal()}'
''')

In [110]:
np.array_split(list(set([1, 2, 3, 4])), 2)

[array([1, 2]), array([3, 4])]

In [98]:
df = main.merge(mart_orders, how='left', on='order_id') \
    .groupby('device_id').agg({'has_'+e: 'nunique' for e in events} 
                              | {e: max for e in metrincs_order}
                              ).reset_index() \
    .fillna(0)
df.head(2)

,device_id,has_main.pageview,has_listing.pageview,has_registration.pageview,has_experience.pageview,has_booking-calendar.pageview,has_booking.pageview,has_booking:order-create.success,is_confirmed,full_price,delta_hours_order_to_reject,money_refund_amount_rub,comission_tripster_final
0,00009ba8-d3c6-4425-a7f8-017ec581b8f2,1,1,0,1,0,0,0,0,0.0,0.0,0.0,0.0
1,000214f0-bd91-4c88-be04-9f0e619b7439,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0


In [99]:
METRICS = ['has_'+e for e in events] + [e for e in metrincs_order]
METRICS

['has_main.pageview',
 'has_listing.pageview',
 'has_registration.pageview',
 'has_experience.pageview',
 'has_booking-calendar.pageview',
 'has_booking.pageview',
 'has_booking:order-create.success',
 'is_confirmed',
 'full_price',
 'delta_hours_order_to_reject',
 'money_refund_amount_rub',
 'comission_tripster_final']

In [100]:
df['AB'] = rnd.choices(['A', 'B'], k=len(df))

In [105]:
x

AB,A,B,delta,pv
has_listing.pageview,0.53608,0.53699,+0.2%,0.62
has_registration.pageview,0.01395,0.01380,-1.0%,0.739
has_experience.pageview,0.49352,0.49420,+0.1%,0.71
has_booking-calendar.pageview,0.24472,0.24432,-0.2%,0.799
has_booking.pageview,0.24472,0.24432,-0.2%,0.799
has_booking:order-create.success,0.11503,0.11471,-0.3%,0.791
is_confirmed,0.04138,0.04074,-1.6%,0.377
full_price,743.70886,722.36440,-2.9%,0.325
delta_hours_order_to_reject,7.03391,7.14234,+1.5%,0.598
money_refund_amount_rub,14.22711,13.71977,-3.6%,0.604


In [102]:
df['AB'] = rnd.choices(['A', 'B'], k=len(df))
calculus(df)

devcieA 146691 deviceB 146413
Index(['A', 'B', 'pv', 'delta'], dtype='object', name='AB')


AB,A,B,delta,pv
has_listing.pageview,0.53546,0.53761,+0.4%,0.243
has_registration.pageview,0.01405,0.01370,-2.5%,0.419
has_experience.pageview,0.49331,0.49441,+0.2%,0.551
has_booking-calendar.pageview,0.24455,0.24449,+-0.0%,0.969
has_booking.pageview,0.24455,0.24449,+-0.0%,0.969
has_booking:order-create.success,0.11509,0.11466,-0.4%,0.715
is_confirmed,0.04059,0.04153,+2.3%,0.198
full_price,729.57122,736.46620,+0.9%,0.75
delta_hours_order_to_reject,6.90873,7.26808,+5.2%,0.08
money_refund_amount_rub,12.85692,15.09107,+17.4%,0.022


In [103]:
df['AB'] = rnd.choices(['A', 'B'], k=len(df))
calculus(df)

devcieA 146261 deviceB 146843
Index(['A', 'B', 'pv', 'delta'], dtype='object', name='AB')


AB,A,B,delta,pv
has_listing.pageview,0.53608,0.53699,+0.2%,0.62
has_registration.pageview,0.01395,0.01380,-1.0%,0.739
has_experience.pageview,0.49352,0.49420,+0.1%,0.71
has_booking-calendar.pageview,0.24472,0.24432,-0.2%,0.799
has_booking.pageview,0.24472,0.24432,-0.2%,0.799
has_booking:order-create.success,0.11503,0.11471,-0.3%,0.791
is_confirmed,0.04138,0.04074,-1.6%,0.377
full_price,743.70886,722.36440,-2.9%,0.325
delta_hours_order_to_reject,7.03391,7.14234,+1.5%,0.598
money_refund_amount_rub,14.22711,13.71977,-3.6%,0.604


In [7]:
q.query('order_id > ""')

,device_id,order_id,main.pageview,listing.pageview,booking:order-create.success
21,dafce770-9b12-4b79-ac70-9d115fadad81,5391944,1,1,1
26,cb086cd9-59f4-4a01-9c06-2d53550859a1,5384788,1,0,1
36,33a1285f-17fc-4c10-b9b2-9a31dfe6c609,5386583,1,1,1
46,635e03c4-4e60-4dfb-b745-97f64827a263,5384561,1,1,1
56,6b9567bf-a112-4538-af8d-b8be3e9f0208,5384696,1,1,1
...,...,...,...,...,...
51116,4fdbef92-3c65-4970-889b-2774a5d44db7,5400107,1,1,1
51122,dafce770-9b12-4b79-ac70-9d115fadad81,5392892,1,1,1
51126,c012ef28-d07e-48fd-95d7-d83fad06ed13,5390492,1,1,1
51127,d83d0a7e-ebc2-47eb-8d51-aa618cedf124,5387433,1,1,1


,device_id,order_id,main.pageview,listing.pageview,booking:order-create.success
0,b21e37f3-5ff0-4f58-876c-716b1f3fad53,,1,0,0
1,8f459c9c-5eb1-4a5e-9c79-c2aabf6df391,,1,1,0
2,c978cea3-33db-4f00-8752-69a1e39b42b5,,1,1,0
3,a46e58f4-5b60-48e2-8c68-8a95b61f3b0a,,1,1,0
4,45a460d7-ad00-489c-a278-e0172c466a7d,,1,1,0
...,...,...,...,...,...
51145,0ab4b053-afaf-44ce-8643-43a08900fa9e,,1,0,0
51146,ce927087-920e-44f3-a058-a8db82c6f71e,,1,0,0
51147,e5e859dc-9e33-40cf-b1e7-67a906d125e8,,1,1,0
51148,6b526302-2c8d-47b4-931b-15bbd37a0299,,1,0,0


In [ ]:
def get_gr_main1(events):
    print('ord')
    ord = sorted(events['order_id'].unique())[1:]

    orders = pd.concat([anl(f'''
        select order_id, traveler_id, traveler_email, payment_date, money_refund_date, order_creation_platform, is_event_ended, exp_type, exp_type_id, exp_format, order_status
        from analytics.mart_orders 
        where order_id in ({literal(batch)})
        ''')
        for batch in np.array_split(ord, 2)
    ])
    orders['zabron'] = orders['order_status'] == "Забронировано"
    orders['order_id'] = orders['order_id'].astype(str)

    main = events.merge(orders, how='left', on='order_id')

    main_gr = main.groupby('device_id') \
        .agg({
            'has_listing.pageview': 'max',
            'has_booking:order-create.success': 'max',
            'has_main.pageview': 'max',
            'zabron': 'max',     
        }).reset_index() \
        .fillna(False)
    main_gr['AB'] = np.random.binomial(1, 0.5, size=len(main_gr))
    main_gr1 = main_gr.query('AB == 1') \
        .agg({'device_id': 'count', 
              'has_booking:order-create.success': 'sum', 
              'zabron': 'sum', })
    return main_gr1

def mdeshnik(gr_main):
    n = gr_main['device_id']
    ord = gr_main['has_booking:order-create.success']
    zabron = gr_main['zabron']
    display(pd.DataFrame({'A': gr_main}).T)
    print('devices', gr_main['device_id'], 'main2order', round(ablib.mde_ratio(ord, n), 1), 'main2zabron', round(ablib.mde_ratio(zabron, n), 1), )


In [ ]:
start_dt = '2025-08-01'
finis_dt = '2025-08-02'

def big_func(start_dt, finis_dt):
    events = sogu_filter_has_events('''event_name = 'main.pageview' ''', ['main.pageview', 'listing.pageview', 'booking:order-create.success'], start_dt, finis_dt)

    gr_main1 = get_gr_main1(events)

    mdeshnik(gr_main1)


In [ ]:
big_func('2025-08-01', '2025-08-07')

In [ ]:
big_func('2025-08-01', '2025-08-14')

In [ ]:
big_func('2025-08-01', '2025-08-21')

In [ ]:
big_func('2025-08-01', '2025-08-21')

In [ ]:
46_841 / (46_841 + 47_858)

In [ ]:
46_841 + 47_858

In [ ]:
n = 46_841 + 47_858
k = 46_841
p1 = k / n
p = 0.5
A = np.random.binomial(n, 0.5, size=100_000) / n
plt.axvline(p1)
plt.hist(A, bins=40);

In [ ]:
16719 / 146544

In [ ]:
n0 = 105_000
p0 = 0.1083
p1 = p0 * (1 + 0.035)

print(p0, p1)
print(round(p0*n0), round(p1*n0))
A = np.random.binomial(n0, p0, size=1_000_000) / n0
B = np.random.binomial(n0, p1, size=1_000_000) / n0

perc95 = np.percentile(A, 95)
perc20 = np.percentile(B, 20)

print(95, perc95)
print(20, perc20)

plt.axvline(perc95, color='black')
plt.hist(A, color='green', bins=30)
plt.hist(B, color='red', bins=30, alpha=0.7)
plt.legend();

In [ ]:
proportions_ztest([16719, 17020], [n0, n0])[1]

In [ ]:
page_view = sogu('''
    select
        toDate(dt) d
        ,uniqExact(device_id) cnt
    {FROM_SOGU_WHERE_DT}
        and event_name = 'main.pageview' 
    group by d 
    order by d asc;
''')
plt.plot(page_view.d, page_view.cnt)

In [ ]:
events = sogu(f'''
    with devices as (
        select distinct 
            device_id
            {FROM_SOGU_WHERE_DT}
            and event_name = 'main.pageview'     
    )
    select
        device_id
        ,toDate(dt) as day
        ,event_name
        ,JSONExtractString(params, 'order_id') as order_id
    {FROM_SOGU_WHERE_DT}
        and event_name in ('main.pageview', 'listing.pageview', 'booking:order-create.success')
        and device_id in devices
    
''')
for ev in events.event_name.unique():
    print(ev)
    devises = set(events[events.event_name == ev].device_id)
    events['has_'+ev] = events['device_id'].isin(devises)# * events['device_id']

In [ ]:
ord = sorted(events['order_id'].unique())[1:]

orders = pd.concat([anl(f'''
    select {mart_orders_columns.core}
    from analytics.mart_orders 
    where order_id in ({literal(batch)})
    ''')
    for batch in tqdm(np.array_split(ord, 3))
])
orders['zabron'] = orders['order_status'] == "Забронировано"
orders['order_id'] = orders['order_id'].astype(str)
print(len(orders))
orders.head(2)

In [ ]:
main = events.merge(orders, how='left', on='order_id')
print(len(main))
display(main.head(2))

In [ ]:
main_gr = main.groupby('device_id') \
    .agg({
        'has_listing.pageview': 'max',
        'has_booking:order-create.success': 'max',
        'has_main.pageview': 'max',
        'zabron': 'max',     
    }).reset_index() \
    .fillna(False)
print(len(main_gr))
main_gr.head(2)

In [ ]:
def power_ratio(k0, n0, mde):
    p0 = k0 / n0
    p1 = p0 * (1 + mde/100)
    A = stats.binom(n0, p0)
    B = stats.binom(n0, p1)
    A_perc95 = A.ppf(0.95)2
    return 1 - B.cdf(A_perc95)

power(400, 1000, 9.8)

In [ ]:
def mde_ratio(k0, n0, alpha=0.05, beta=0.20):
    mde=50
    delta_mde = 25
    for _ in tqdm(range(18)): # 18 шагов достаточно
        pow = power(k0, n0, mde)
        dlt = pow - (1-beta)
        mde += delta_mde * (-1 if dlt > 0 else +1)
        delta_mde *= 2/3
    return mde
mde(400, 1000)

In [ ]:
p0 = 0.4
p1 = p0 * 1.1
n0 = 1000

print(p0, p1)

A = np.random.binomial(n0, p0, size=10_000)
B = np.random.binomial(n0, p1, size=10_000)
print(np.percentile(A, 95))
print(np.percentile(B, 20))
plt.hist(A, color='green', bins=38)
plt.hist(B, color='red', bins=38);

In [ ]:
ll = []

for _ in range(1000):
    a = rnd.choice(A)
    b = rnd.choice(B)
    q = proportions_ztest([a, b], [n0, n0])[1]
    ll.append(q) 

plt.hist(ll, bins=20);
np.percentile(ll, 80)

In [ ]:
def power(k0, n0, up, alpha, n_size=100_000):
    p0 = k0 / n0
    p1 = p0*(1+up/100)
    print(p0, p1)
    # pvalue > alpha - это ошибка второго рода
    # проводим n_size виртуальных экспериментов, считаем количество ошибок второго рода
    return 1 - sum([proportions_ztest([np.random.binomial(n0, p0), 
                                   np.random.binomial(n0, p1)], 
                                  [n0, n0])[1] > alpha
                
                for _ in tqdm(range(n_size))
            ]) / n_size

power(400, 1000, 18.9, 0.05)

In [ ]:
def mde_find(k0, n0, up=50, alpha=0.05, beta=0.20):
    lr_rate = 0.2
    while True:
        pow = power(k0, n0, up, alpha)
        dlt = pow - beta
        print(up, pow, dlt, dlt/beta)
        up = up * (1 - dlt*1.2)

mde_find(40, 100)
        

In [ ]:
np.random.binomial(10, 0.4, 6) / 10

In [ ]:
main_gr.groupby('zabron', dropna=False).agg({'device_id': 'count'})

In [ ]:
main_gr.agg({m: 'mean' for m in ['has_listing.pageview', 'has_booking:order-create.success', 'has_main.pageview', 'zabron']})

In [ ]:
0.088567 / 0.118583

In [ ]:
orders.agg({'zabron': 'mean', '': ''})

In [ ]:
orders['order_status'].unique()

In [ ]:
uniq = list(events.device_id.unique())
rnd.random.seed(0)
AB = pd.DataFrame({
    'device_id': uniq,
    'AB': [ rnd.randint(0, 1) for _ in range(len(uniq)) ]
})
AB

In [ ]:
events

In [ ]:
delta_days = 
sogu('''
    select 
    from sogu.events 
    where toDate(dt) between '2025-08-0
''')

In [ ]:
sogu('select 1')

In [ ]:
Stage = namedtuple('Stage', ['randomization_unit_id', 'database', 'table', 'condition'])
Stage(10, 20, 30, 40).database

In [ ]:
DEVICE_ID2MAIN_PAGEVIEW__metric = "DEVICE_ID", 'conversion', "MAIN_PAGEVIEW",
DEVICE_ID2SOGU_ORDER_CREATED__metric = "DEVICE_ID", 'conversion', "SOGU_ORDER_CREATED",
DEVICE_ID2LISTING_PAGEVIEW__metric = "DEVICE_ID", 'conversion', "LISTING_PAGEVIEW",
ORDER2REFUND = "DEVICE_ID", 'conversion', ""

In [ ]:
def craft_sql_from_metric(metric):
    '''
        Создаёт SQL для вычисления одной метрики
    '''

    denom__stage = stages[metric[0]]
    nom__stages = stages[metric[2]]

    query = f'''
    select 
        denom_table.{denom__stage.randomization_unit_id} as {metric[0]}
        ,uniqExactIf(denom_table.{denom__stage.randomization_unit_id}, {nom__stage.condition}) as {metric[2]}_{metric[1]}
    {FROM_SOGU_WHERE_DT}
        and ({denom__stage[3]}
            or {nom__stage[3]})
    group by  {metric[0]}
    having count({denom__stage[3]}) > 0
    '''
    return query
st = craft_sql_from_metric(DEVICE_ID2MAIN_PAGEVIEW__metric)
for e in st.split('\n'):
    print(e)

In [ ]:
def craft_sql_from_many_metrics(metrics_list):
    '''
        Создаёт SQL для вычисления группы метрик с одним знаменателем
    '''
    assert sum([m[0] == metrics_list[0]  for m in metrics_list]) == len(metrics_list), 'У метрик разный знаменатель. Используйте общий'
    denom__stage = stages[metrics_list[0][0]]

    nom__stages_list = [stages[m[2]] for m in metrics_list]

    query = f'''
        select 
            se.{denom__stage.randomization_unit_id} as {metric[0]}
            {[ f",uniqExactIf(se.{denom__stage.randomization_unit_id}, {nom__stage.condition}) as {metric[2]}_{metric[1]}"
                    for metric in 
            ]}
            
            ,uniqExactIf(se.)
        {FROM_SOGU_WHERE_DT}
        group by  {metric[0]}
        having count({denom__stage[3]}) > 0
    '''
    return query
st = craft_sql_from_metric([DEVICE_ID2MAIN_PAGEVIEW__metric, DEVICE_ID2LISTING_PAGEVIEW__metric, DEVICE_ID2SOGU_ORDER_CREATED__metric])
for e in st.split('\n'):
    print(e)

In [ ]:
sogu('''
     select 
            denom_table.device_id as DEVICE_ID
            ,uniqExactIf(denom_table.device_id,  event_name = 'main.pageview' ) as MAIN_PAGEVIEW_conversion
        from sogu.events as denom_table
        where  toDate(dt) between toDate('2025-08-10 13:00:00') and toDate('2025-08-12 14:15:16')
            and dt between '2025-08-10 13:00:00' and '2025-08-12 14:15:16'
            and device_id <> ''
            and ( event_name = 'experiments.hit' 
                or  event_name = 'main.pageview' )
        group by  DEVICE_ID
        having count( event_name = 'experiments.hit' ) > 0
     ''')

In [ ]:
df = sogu(st)
print(len(df))
df.head()

In [2]:
new_experiences = list(anl('''
select id as experience_id
from tripster.core_experience
where created_on >= '2025-08-15'
''')['experience_id'])
print(new_experiences)

[99344, 99199, 99412, 100965, 101445, 100322, 99120, 99730, 100538, 102016, 99672, 102196, 99163, 101065, 101177, 99621, 99989, 101381, 102049, 100353, 99164, 98881, 101311, 100339, 101534, 101093, 99859, 100009, 98747, 100780, 100407, 99251, 101689, 101629, 100183, 100374, 101837, 102102, 99902, 100611, 99755, 99832, 99346, 100697, 101711, 101575, 102398, 102044, 102104, 101683, 99860, 100327, 99900, 99731, 101211, 99220, 100312, 101216, 100595, 102053, 100687, 100781, 99318, 100799, 100328, 99290, 98925, 99813, 100722, 98953, 99661, 100172, 101160, 100326, 100743, 99368, 100501, 100470, 100503, 100467, 101536, 99890, 99141, 99147, 99901, 100745, 102042, 100621, 99990, 98868, 102051, 100333, 101075, 102052, 99651, 100290, 102045, 101097, 99775, 100331, 100740, 102014, 99347, 100782, 101976, 101790, 101788, 100068, 101535, 102364, 101385, 99121, 102054, 102056, 101240, 100039, 102023, 100036, 101718, 101789, 99221, 100721, 100746, 102085, 100414, 100623, 100700, 101098, 100037, 100597,

In [5]:
len(new_experiences)

3755

In [7]:
main = sogu(f'''
with orders as (
    select distinct 
        device_id
        ,JSONExtractInt(params, 'order_id') as order_id
            
    from sogu.events as se
    where toDate(se.dt) between toDate('2025-09-15') and toDate('2025-10-13')
        and se.dt between '2025-09-15' and '2025-10-13'
        and se.device_id <> ''
        and ua_is_bot = 0 

        and event_name = 'booking:order-create.success'
)
,limitation as (
	select device_id 
	 
    from sogu.events as se
    where toDate(se.dt) between toDate('2025-09-15') and toDate('2025-10-13')
        and se.dt between '2025-09-15' and '2025-10-13'
        and se.device_id <> ''
        and ua_is_bot = 0 
        and event_name in ('experiments.hit', 'listing.pageview')
        and if(event_name = 'experiments.hit', JSONExtractString(params, 'experiment') = 'ranking_rpv_new', true)
    group by device_id
    having anyIf(1, event_name = 'experiments.hit')
        and anyIf(1, event_name = 'listing.pageview')
)

select 
    device_id
    ,maxIf(JSONExtractString(params, 'variant'), event_name = 'experiments.hit' and JSONExtractString(params, 'experiment') = 'ranking_rpv_new') as ab_variant_min
    ,minIf(JSONExtractString(params, 'variant'), event_name = 'experiments.hit' and JSONExtractString(params, 'experiment') = 'ranking_rpv_new') as ab_variant_max
    ,ab_variant_min as ab_variant
    ,anyIf(1,  event_name = 'listing.pageview' ) as "has_listing_pageview"
    ,anyIf(1,  event_name = 'experience.pageview' ) as "has_experience_pageview"
    ,anyIf(1,  event_name = 'booking.pageview' ) as "has_booking_pageview"
    ,anyIf(1,  event_name = 'booking:order-create.success' ) as "has_sogu_order_created"
            
    ,anyIf(1, event_name = 'experience.pageview' and JSONExtractString(params, 'experience_id') in ({literal(new_experiences)}) ) as new_experience_view
    ,anyIf(1, event_name = 'booking.pageview' and JSONExtractString(params, 'experience_id') in ({literal(new_experiences)}) ) as new_experience_booking_view
    ,anyIf(1, event_name = 'booking:order-create.success' and JSONExtractString(params, 'experience_id') in ({literal(new_experiences)}) ) as new_order_success

from sogu.events as se
where toDate(se.dt) between toDate('2025-09-15') and toDate('2025-10-13')
    and se.dt between '2025-09-15' and '2025-10-13'
    and se.device_id <> ''
    and ua_is_bot = 0 

    and device_id in limitation
    and event_name in ('experiments.hit', 'listing.pageview', 'experience.pageview', 'booking.pageview', 'booking:order-create.success')
group by device_id
having ab_variant_min = ab_variant_max
''')
main

,device_id,ab_variant_min,ab_variant_max,ab_variant,has_listing_pageview,has_experience_pageview,has_booking_pageview,has_sogu_order_created,new_experience_view,new_experience_booking_view,new_order_success
0,6ce1442f-8837-4dd0-82d0-157d78751f80,original,original,original,1,0,0,0,0,0,0
1,117b726b-7403-47b4-9d78-6c02fcce28a8,ranking_rpv,ranking_rpv,ranking_rpv,1,1,1,0,0,0,0
2,122f5064-381b-4a17-9016-8a2253028f0a,ranking_mab_new,ranking_mab_new,ranking_mab_new,1,1,0,0,0,0,0
3,79eb21f2-7052-4cbb-9b48-7acd46156c37,ranking_rpv,ranking_rpv,ranking_rpv,1,0,0,0,0,0,0
4,f6dc29c7-3c3f-49a4-b991-fb9141bb9cdc,ranking_mab_new,ranking_mab_new,ranking_mab_new,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1041900,8f199e41-42c5-4c31-b763-77c7732a67b8,original,original,original,1,1,1,0,0,0,0
1041901,f36da271-993c-429c-a878-e50ce9a87979,original,original,original,1,0,0,0,0,0,0
1041902,8c58e3d4-4cc4-4838-b595-217a616f474b,ranking_rpv,ranking_rpv,ranking_rpv,1,0,0,0,0,0,0
1041903,e299593f-5134-4060-917d-d320967fde28,ranking_rpv,ranking_rpv,ranking_rpv,1,0,0,0,0,0,0


In [10]:
main.groupby('ab_variant').agg({ 
    'has_experience_pageview': 'mean',
    'has_booking_pageview': 'mean',
    'has_sogu_order_created': 'mean',
    'new_experience_view': 'mean',
    'new_experience_booking_view': 'sum',
    'new_order_success': 'sum',
})

,has_experience_pageview,has_booking_pageview,has_sogu_order_created,new_experience_view,new_experience_booking_view,new_order_success
ab_variant,,,,,,
original,0.434863,0.143875,0.046217,0.024176,1496,296
ranking_mab_new,0.435795,0.143112,0.045494,0.047169,2934,519
ranking_rpv,0.434189,0.141806,0.045049,0.029929,1813,310


In [11]:
1813 / 1496 - 1

0.21189839572192515

In [4]:
sogu('select * from sogu.events limit 2')['experience_id']

KeyError: 'experience_id'